In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
# import import_ipynb
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
import os

In [ ]:
describe notebook notebook_test_a

In [ ]:
# remember to add IPython and nbformat to the notebook packages
import os, sys, types
from nbformat import read
from snowflake.snowpark import Session
from IPython.core.inputtransformer2 import TransformerManager
input_transformer_manager = TransformerManager()
_session = Session.builder.getOrCreate()
class NotebookLoader(object):
    """Module Loader for Snowflake Notebooks"""
    def __init__(self, path=None):
        self.path = path
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        print ("importing notebook from %s" % path)
        f = _session.file.get_stream(path)
        # load the notebook object
        nb = read(f, 4)
        # create the module and add it to sys.modules if name in sys.modules:
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        sys.modules[fullname] = mod
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            pass
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname)
        if not nb_path:
            return
        key = path
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

def find_notebook(fullname, path=None):
    try:
        notebook_root = _session.sql(f"describe notebook {fullname}").select('"live_version_location_uri"').collect()[0][0]
        notebook_name = _session.sql(f"describe notebook {fullname}").select('"main_file"').collect()[0][0]
        notebook_location = '/tmp/appRoot/' + notebook_name
        print(f"Location of the notebooks is: {notebook_location}")
        return notebook_location
    except:
        return ""

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self,database:str=None,schema:str=None):
        self.loaders = {}
        self.database = database
        self.schema = schema
    def find_module(self, fullname, path=None):
        if self.database and self.schema:
            fullname = self.database + "." + self.schema + "." + fullname
        print(f"Finding {fullname}")
        nb_path = find_notebook(fullname)
        if not nb_path:
            return
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [ ]:
sys.meta_path.append(NotebookFinder())

In [ ]:
import notebook_test_a